In [ ]:
import os
import zipfile
from PIL import Image
import numpy as np
from io import BytesIO
# test
from zip_data_loader import load_images_as_list_of_dicts, test_format

test_format()

ZIP: 14-7-35.zip has 20 images
  Image 1 shape: (3024, 4032, 3)
  Image 2 shape: (3024, 4032, 3)
  Image 3 shape: (3024, 4032, 3)
  Image 4 shape: (3024, 4032, 3)
  Image 5 shape: (3024, 4032, 3)
  Image 6 shape: (3024, 4032, 3)
  Image 7 shape: (3024, 4032, 3)
  Image 8 shape: (3024, 4032, 3)
  Image 9 shape: (3024, 4032, 3)
  Image 10 shape: (3024, 4032, 3)
  Image 11 shape: (3024, 4032, 3)
  Image 12 shape: (3024, 4032, 3)
  Image 13 shape: (3024, 4032, 3)
  Image 14 shape: (3024, 4032, 3)
  Image 15 shape: (3024, 4032, 3)
  Image 16 shape: (3024, 4032, 3)
  Image 17 shape: (3024, 4032, 3)
  Image 18 shape: (3024, 4032, 3)
  Image 19 shape: (3024, 4032, 3)
  Image 20 shape: (3024, 4032, 3)
ZIP: 15-15-15.zip has 20 images
  Image 1 shape: (3024, 4032, 3)
  Image 2 shape: (3024, 4032, 3)
  Image 3 shape: (3024, 4032, 3)
  Image 4 shape: (3024, 4032, 3)
  Image 5 shape: (3024, 4032, 3)
  Image 6 shape: (3024, 4032, 3)
  Image 7 shape: (3024, 4032, 3)
  Image 8 shape: (3024, 4032, 3)
  